In [1]:
import pandas as pd
import numpy as np
import scipy as sp 
import matplotlib as plt
import matplotlib.pyplot as pltp
import plotly.plotly as py 
import missingno as msno
from statsmodels.graphics.mosaicplot import mosaic
import seaborn as sns
from collections import Counter
from __future__ import print_function
%matplotlib inline
pd.set_option('max_rows',300)
pd.set_option('max_columns',200)

In [8]:
xl = pd.ExcelFile('Complaint_Data_Scraped.xlsx')
sheetnames = xl.sheet_names  # see all sheet names
sheetnames

[u'Panel complaint data scraped',
 u'EIB complaint data scraped',
 u'MICI complaint data scraped',
 u'CAO complaint data scraped',
 u'ADB complaint data scraped']

In [15]:
dfs = {}
for name in sheetnames:
    df = xl.parse(name)
    dfs[name.split()[0]] = df
    print(name.split()[0],df.shape)
    print(df.columns,'\n','*'*40)


Panel (122, 31)
Index([u'IAM', u'Year', u'Country', u'Project', u'ID', u'IAM ID', u'Filer(s)',
       u'Environmental Category', u'Project Company', u'Project Number',
       u'Related Project Number', u'Project Type', u'Financial Institution',
       u'Project Loan Amount', u'Sector', u'Issues', u'Complaint Status',
       u'Filing Date', u'Registration Start Date', u'Registration End Date',
       u'Eligibility Start Date', u'Eligibility End Date',
       u'Dispute Resolution Start Date', u'Dispute Resolution End Date',
       u'Compliance Review Start Date', u'Compliance Review End Date',
       u'Monitoring Start Date', u'Monitoring End Date', u'Date Closed',
       u'Documents', u'Hyperlink'],
      dtype='object') 
 ****************************************
EIB (197, 31)
Index([u'IAM', u'Year', u'Country', u'Project', u'ID', u'IAM ID', u'Filer(s)',
       u'Environmental Category', u'Project Company', u'Project Number',
       u'Related Project Number', u'Project Type', u'Financia

** Inital Thoughts**

Why don't they have the same columns, ADB only has 7, MICI has a 'Hyperlink' and 'Hyperlink [Correct]', need to check that this is unaltered output from the scrapers. 

Other thoughts for doing comparison
1. Do we want them to be aggregated into a single file for comparison?  or just compare to the output files per scraper ? 
2. At the DataDive should we have the teams just work with one scrapers output, and then a fake set of altered data ... this might make more sense. 
3. Need to figure out how to create fake changed data. 

,IAM,Year,Country,Project,ID,IAM ID,Filer(s),Environmental Category,Project Company,Project Number,Related Project Number,Project Type,Financial Institution,Project Loan Amount,Sector,Issues,Complaint Status,Filing Date,Registration Start Date,Registration End Date,Eligibility Start Date,Eligibility End Date,Dispute Resolution Start Date,Dispute Resolution End Date,Compliance Review Start Date,Compliance Review End Date,Monitoring Start Date,Monitoring End Date,Date Closed,Documents,Hyperlink
0,Panel,2017,Cameroon,Lom Pangar Hydropower Project,122,23,Two former workers of the Lom Pangar Hydropowe...,A,NaN,P114077,NaN,NaN,IDA,NaN,NaN,NaN,Open,Completed,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://ewebapps.worldbank.org/apps/ip/Pages/Vi...
1,Panel,2017,Peru,Boosting Human Capital and Productivity Develo...,121,23,"Edwin Montenegro, President of the Organizació...",NaN,NaN,P156858,NaN,NaN,IBRD,NaN,NaN,NaN,Open,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Completed,NaN,http://ewebapps.worldbank.org/apps/ip/Pages/Vi...
2,Panel,2017,"Congo, Democratic Republic of",DRC High Priority Roads Reopening and Maintena...,120,23,Community members residing in the project area...,A,NaN,P153836,NaN,NaN,IDA,NaN,NaN,NaN,Open,NaN,NaN,NaN,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://ewebapps.worldbank.org/apps/ip/Pages/Vi...
3,Panel,2017,India,Amaravati Sustainable Capital City Development...,119,23,The Requesters are landowners in the area prop...,A,NaN,P159808,NaN,NaN,IBRD,NaN,NaN,NaN,Open,NaN,NaN,NaN,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://ewebapps.worldbank.org/apps/ip/Pages/Vi...
4,Panel,2017,Vietnam,Livestock Competitiveness and Food Safety(P090...,118,23,The Requesters are two animal-protection organ...,B,NaN,P090723 and P151946,NaN,NaN,IDA,NaN,NaN,NaN,Open,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Completed,NaN,http://ewebapps.worldbank.org/apps/ip/Pages/Vi...
5,Panel,2017,Serbia,Floods Emergency Recovery Project (P152018),117,23,The Requesters are from Serbia and say they li...,B,NaN,P152018,NaN,NaN,IBRD,NaN,NaN,NaN,Open,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Completed,NaN,http://ewebapps.worldbank.org/apps/ip/Pages/Vi...
6,Panel,2016,Bangladesh,Bangladesh Modern Food Storage Facilities Proj...,116,23,The Requesters are from Bangladesh and say the...,B,NaN,P120583,NaN,NaN,IDA,NaN,NaN,NaN,Open,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Completed,NaN,http://ewebapps.worldbank.org/apps/ip/Pages/Vi...
7,Panel,2016,Kenya,Water and Sanitation Service Improvement Proje...,115,23,The Requesters are from Kenya and say they liv...,A,NaN,P096367 and P126637,NaN,NaN,IDA,NaN,NaN,NaN,Open,NaN,NaN,NaN,NaN,Completed,NaN,NaN,NaN,NaN,NaN,NaN,Completed,NaN,http://ewebapps.worldbank.org/apps/ip/Pages/Vi...
8,Panel,2016,India,Proposed Amaravati Sustainable Capital City De...,114,23,The Requesters state they live in the area kno...,A,NaN,P159808,NaN,NaN,IBRD,NaN,NaN,NaN,Open,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Completed,NaN,http://ewebapps.worldbank.org/apps/ip/Pages/Vi...
9,Panel,2016,Uganda,"Private Power Generation (Bujagali), Water Man...",113,23,The Request for Inspection was submitted by me...,B,NaN,"P123204, P089659, P133312",NaN,NaN,IDA,NaN,NaN,NaN,Open,NaN,NaN,NaN,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://ewebapps.worldbank.org/apps/ip/Pages/Vi...
